In [18]:
import os
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import myServices as ms
import randForest as rfr
import numpy as np
import pandas as pd
import sklearn
import joblib

In [ ]:
# to compute ececution time do:
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

## Random Forest Section

In [ ]:
sklearn.metrics.get_scorer_names()

In [17]:
print(np.linspace(28,60, 19).astype(int))

[28 29 31 33 35 36 38 40 42 44 45 47 49 51 52 54 56 58 60]


In [19]:
DS = pd.read_csv('basin1CleanToTrain.csv', index_col = None)

In [20]:
print(DS.keys())
DS.info()
print(DS.keys())


Index(['percentage', 'disToRiv', 'TWI', 'TPI', 'FAcc', 'slope', 'elevation'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479538 entries, 0 to 2479537
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   percentage  int64  
 1   disToRiv    float64
 2   TWI         float64
 3   TPI         float64
 4   FAcc        float64
 5   slope       float64
 6   elevation   float64
dtypes: float64(6), int64(1)
memory usage: 132.4 MB
Index(['percentage', 'disToRiv', 'TWI', 'TPI', 'FAcc', 'slope', 'elevation'], dtype='object')


In [23]:
X, Y = ms.importDataSet('basin1CleanToTrain.csv', 'percentage')
classCount = Counter(Y)
print(classCount)

Counter({'percentage': 1})


In [29]:
print(np.unique(Y, return_counts=True))

(array([0, 1, 5]), array([2426656,    9062,   43820]))


In [35]:
from numpy import where
print(where(Y == 1)[0])

[   2639    2818    3000 ... 2454493 2454511 2455177]
